<a href="https://colab.research.google.com/github/VellummyilumVinoth/FineTune_ALBERT_NLP/blob/main/FineTune_ALBERT_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00


In [2]:
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch
import tensorflow as tf
import os
import numpy as np
import re

# Load the ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [3]:
with open('The Hunger Games.txt','r') as f:
    file = f.read().split(".")

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

text_lst = []
len_lst = []
for line in file:
    if len(line.split(" ")) >=20:
        line = clean_text(line)
        text_lst.append(line)
        len_lst.append(len(line.split(" ")))


In [4]:
text_lst

[' Even though it was years ago, I think he still remembers how I tried to drown him in a bucket when Prim brought him home.',
 ' I pull on trousers, a shirt, tuck my long dark braid up into a cap, and grab my forage bag.',
 ' On the table, under a wooden bowl to protect it from hungry rats and cats alike, sits a perfect little goat cheese wrapped in basil leaves.',
 ' Our part of District , nicknamed the Seam, is usually crawling with coal miners heading out to the morning shift at this hour.',
 ' Men and women with hunched shoulders, swollen knuckles, many who have long since stopped trying to scrub the coal dust out of their broken nails, the lines of their sunken faces.',
 ' Separating the Meadow from the woods, in fact enclosing all of District , is a high chain link fence topped with barbed wire loops.',
 ' In theory, its supposed to be electrified twenty four hours a day as a deterrent to the predators that live in the woods packs of wild dogs, lone cougars, bears that used to t

In [5]:
train_size = int(len(text_lst) * 0.8)
train_size

1253

In [6]:
train_dataset = text_lst[:train_size]
test_dataset = text_lst[train_size:]

In [7]:
train_dataset

[' Even though it was years ago, I think he still remembers how I tried to drown him in a bucket when Prim brought him home.',
 ' I pull on trousers, a shirt, tuck my long dark braid up into a cap, and grab my forage bag.',
 ' On the table, under a wooden bowl to protect it from hungry rats and cats alike, sits a perfect little goat cheese wrapped in basil leaves.',
 ' Our part of District , nicknamed the Seam, is usually crawling with coal miners heading out to the morning shift at this hour.',
 ' Men and women with hunched shoulders, swollen knuckles, many who have long since stopped trying to scrub the coal dust out of their broken nails, the lines of their sunken faces.',
 ' Separating the Meadow from the woods, in fact enclosing all of District , is a high chain link fence topped with barbed wire loops.',
 ' In theory, its supposed to be electrified twenty four hours a day as a deterrent to the predators that live in the woods packs of wild dogs, lone cougars, bears that used to t

In [8]:
import random

class StatementDataset(torch.utils.data.Dataset):
    def __init__(self, source_statements,tokenizer):
        self.source_statements = source_statements
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.source_statements)
    
    def __getitem__(self, idx):
        source_statements = str(self.source_statements[idx])

        masked_sentences = []
        tokenized_sentence = self.tokenizer.tokenize(source_statements)
        num_words_to_mask = max(1, int(0.15 * len(tokenized_sentence))) # mask at least 1 word
        words_to_mask = random.sample(range(len(tokenized_sentence)), num_words_to_mask)
        masked_sentence = ""
        for i in range(len(tokenized_sentence)):
            if i in words_to_mask:
                masked_sentence += "[MASK]"
            else:
                masked_sentence += tokenized_sentence[i] + " "
        masked_sentences.append(masked_sentence.strip())

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_sentences,
            add_special_tokens=False,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        print("Masked source statement:", masked_sentences)

        return torch.tensor(input_ids)

In [9]:
# train_dataset = tokenizer(train_dataset,max_length=100,truncation=True,padding='max_length',return_tensors='tf')

# print(train_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
# test_dataset = tokenizer(test_dataset,max_length=100,truncation=True,padding='max_length',return_tensors='tf')

# print(test_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [9]:
train_data = StatementDataset(train_dataset,tokenizer)
test_data = StatementDataset(test_dataset,tokenizer)

In [10]:
test_data[6]


Masked source statement: ['▁i ▁have ▁to ▁ bury ▁the ▁real ▁pain ▁because ▁who s [MASK]▁to ▁bet ▁on ▁a ▁tribute [MASK]▁keeps [MASK]ive [MASK]▁over ▁the ▁deaths ▁of ▁her ▁opponents .']


tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 500,                # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 1e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,

)

trainer.train()

# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Masked source statement: ['[MASK]▁the ▁next [MASK]▁hours , ▁i ▁alternate ▁between ▁jogging ▁and ▁walking , ▁putting ▁as ▁much ▁distance ▁as ▁i ▁can [MASK]▁myself ▁and ▁my ▁competitors .']
Masked source statement: ['▁i ▁try ▁to ▁think [MASK]▁telling ▁pee [MASK][MASK]▁give ▁him ▁any ▁possible ▁advantage ▁over ▁me , ▁but ▁i ▁dont ▁see ▁how .']
Masked source statement: ['▁i ▁quickly ▁open [MASK]▁window , ▁to s s ▁the ▁cookies ▁pee tas ▁father ▁gave ▁me ▁out [MASK]▁the ▁train , [MASK]▁slam [MASK]▁glass ▁shut .']
Masked source statement: ['▁and [MASK][MASK]▁fire ▁she ▁was ▁supposed ▁to ▁set ▁although ▁i ▁forgot ▁to [MASK]▁for ▁it ▁last ▁night [MASK]▁the ▁farthest ▁from ▁our ▁site ▁of ▁all .']
Masked source statement: ['▁in [MASK]▁time , ▁ive [MASK]▁and ▁gut ted ▁the ▁animal , ▁leaving [MASK][MASK], ▁feet , ▁tail , ▁skin , ▁and ▁in nard s , ▁under ▁a ▁pile ▁of ▁leaves .']
Masked source statement: ['▁the [MASK]▁is ▁that ▁once ▁a ▁tribute s ▁name ▁has ▁been ▁pulled [MASK]▁the ▁ball , ▁another ▁

Step,Training Loss,Validation Loss


Streaming output truncated to the last 5000 lines.
Masked source statement: ['▁i ▁take ▁the ▁time ▁to ▁re order ▁my ▁supplies [MASK]▁wad ▁up [MASK]▁sleeping ▁bag , ▁and ▁mess ily ▁stuff ▁everything ▁into ▁the ▁backpack [MASK]']
Masked source statement: ['[MASK]▁let ▁him ▁drift ▁off ▁to ▁sleep ▁then ▁and ▁attend ▁to [MASK]▁own ▁needs , ▁wolf ing ▁down ▁a [MASK]▁of ▁gro os ling ▁and ▁roots ▁while [MASK][MASK]▁the ▁daily ▁report ▁in ▁the ▁sky .']
Masked source statement: ['▁maybe ▁i ▁did ▁it ▁for ▁myself , ▁pee [MASK], ▁did ▁you ▁ever ▁think ▁of ▁that [MASK]▁you [MASK]▁the ▁only ▁one ▁who ▁ .']
Masked source statement: ['▁was ▁there ▁some ▁part ▁of [MASK][MASK]▁couldnt [MASK]▁trusting ▁him ▁on ▁the [MASK]▁hand , ▁im ▁relieved ▁that ▁we ▁can ▁stop ▁the ▁pretense ▁of ▁being ▁friends .']
Masked source statement: ['▁but ▁what ▁sort [MASK]▁trap ▁have ▁they ▁laid ▁that ▁requires ▁such ▁dexter ity ▁has ▁so [MASK]▁trigger ▁points ▁why ▁did ▁she ▁squeal [MASK]▁as ▁her ▁hands ▁made [MASK]▁with [MAS

('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/spiece.model',
 './finetuned_albert/added_tokens.json')

In [19]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "The quick brown [MASK] jumps over the lazy dog"


# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores
probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k = torch.topk(probs, k=5)

# Print the top 5 predictions and their probability scores
for i, idx in enumerate(top_k.indices):
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    print(f"Prediction {i+1}: {token}, Probability: {top_k.values[i].item()}")


Prediction 1: ▁guy, Probability: 0.053693439811468124
Prediction 2: ▁joyah, Probability: 0.024960173293948174
Prediction 3: ▁blonde, Probability: 0.018834352493286133
Prediction 4: ▁eye, Probability: 0.01855579763650894
Prediction 5: ▁pony, Probability: 0.018297722563147545
